In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn.linear_model import LinearRegression

In [ ]:
originalTrain = pd.read_csv("./train.csv")
originalTest = pd.read_csv("./test.csv")

In [ ]:
originalTrain.head(10)

In [ ]:
originalTest.head(10)

In [ ]:
originalTrain.describe()